# Import LIbaries

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

pd.set_option("display.max_columns",500)
pd.set_option('display.max_colwidth', -1)

# Read in the Data

In [2]:
#senate district by census tract
senator = pd.read_csv('./data/census-tract-senator.csv')

#legislator, by district, includes party
legislator = pd.read_csv('./data/legislator-database.csv')

#school info, mostly chicago (but also other places)
school = pd.read_csv('./data/school-info.csv')
school.drop('School_Description', axis=1, inplace=True)

#Chicago Public Schools, school quality ratings
#level 1+ thr 2+ (good standing), level 2 (prov), level 3 (intensive support)
qual = pd.read_csv('./data/school-qual.csv')

#CPS student census track for school and student
student = pd.read_csv('./data/student.csv')

In [3]:
#import the 3rd reading Roll Call vote count
vote = {'Althoff':'NV', 'Cunningham':'Y', "Martinez":"N", "Righter":"N", "Anderson":"Y",
         "Fowler":"Y", "McCann": "NV", "Rooney":"N", "Aquino":"Y", "Haine":"NV", "McCarter":"N",
         "Rose":"Y", "Barickman":"N", "Harmon":"Y", "McConchie":"N", "Sandoval":"P", 
         "Bennett":"Y", "Harris":"NV", "McConnaughay":"N", "Schimpf":"Y", "Bertino-Tarrant":"Y",
         "Hastings":"Y", "McGuire": "Y","Silverstein": "Y","Biss":  "Y", "Holmes":"Y" , "Morrison":"Y",
         "Stadelman":"Y", "Bivins": "N", "Hunter":"Y", "Mulroe":"Y","Steans": "NV","Brady": "N", 
         "Hutchinson":"Y", "Muñoz": "P", "Syverson":"N", "Bush":"NV", "Jones": "NV", "Murphy": "Y",
         "Tracy":"N", "Castro":"Y", "Koehler":"Y", "Nybo":"N", "Trotter":"Y", "Clayborne": "Y",
         "Landek": "Y", "Oberweis":"N", "Van Pelt":"N", "Collins":"Y", "Lightford":"Y", "Radogno": "N",
         "Weaver": "N", "Connelly":"N", "Link":"Y", "Raoul":"Y", "J. Cullerton":"Y", "Thomas Cullerton": "Y",
         "Manar": "Y", "Rezin" :"Y" }

In [4]:
#create a legislator and district dictionary to map to 
legis_dict = dict(zip(legislator.District, legislator.Legislator))

#read through vote dict, and lookup name in legislator df and record their vote.
roll_call = []
for i in legislator.Legislator:
    for key, value in vote.items():
        if key in i:
            roll_call.append(value)
legislator['roll_call_vote'] = roll_call

In [5]:
nay_dict = {'N':1, 'Y':0, "NV":0, "P":0}
legislator['target'] = legislator.roll_call_vote.map(nay_dict)

# Exploratory Data Questions
- What Senate Districts have Charter Schools?
- What Senate Districts have students who attend a Charter School?
- What Senate Districts (in Chicago) have "low-performing" schools?

## What senate districts/Senators have charter schools?

In [6]:
#From this database of IL schools, how many of them are Charter schools?
school.School_Type_Rollup.value_counts()
#177 charter schools

Non-selective    692
Public           229
Charter          177
Selective        37 
Non-Selective    5  
Name: School_Type_Rollup, dtype: int64

In [7]:
#filter just charters
charter_schools = school[school.School_Type_Rollup == 'Charter']

#filter where school is open
charter_schools = charter_schools[charter_schools.Year_Closed.isnull()]

In [8]:
#How many charter schools are in each Senate District?
charter_schools.State_Sen_District.value_counts(dropna=False)

 5.0     30
 3.0     18
 2.0     12
 1.0     12
 13.0    10
 16.0    9 
 20.0    7 
 17.0    7 
 11.0    6 
 4.0     5 
NaN      4 
 7.0     4 
 14.0    4 
 6.0     3 
 48.0    2 
 34.0    2 
 15.0    2 
 30.0    1 
 8.0     1 
 27.0    1 
 18.0    1 
 33.0    1 
 57.0    1 
 31.0    1 
 46.0    1 
 19.0    1 
 35.0    1 
 53.0    1 
Name: State_Sen_District, dtype: int64

In [9]:
#why are there NaNs? (4) - Campus_Status == "Not yet open" or "Not Campus"
charter_schools[charter_schools.State_Sen_District.isnull()].District_Name
#All are in Chicago

#Question: what does this Campus Status mean?
charter_schools[charter_schools.State_Sen_District.isnull()]

,School_Info_ID,School_Name,RCDTS,SF_ORG_ID,CPS_ID,Street_Address,City,State,Zip,Phone,Website,Lat,Long,Year_Opened,Year_Closed,District_Name,District_Info_ID,Charter_Renew_YR,School_Type_Rollup,School_Type_Detail,Campus_Status,Network,Community_Area,State_Rep_District,State_Sen_District,Ward,Genders_Served,Serves_ES,Serves_HS,In_Chicago,Year_Reached_Capacity,Serves_MS,ISBE_School_Info_ID,Enrollment_Cap,GEOID10,Utilization_16
959,101313,YCCS Aggregated record,15016299025008C,001U000000NDkrM,609686.0,NaN,Chicago,IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,City of Chicago SD 299,1155,NaN,Charter,Alternative charter,Not Campus,NaN,Douglas,NaN,NaN,3.0,ALL,0.0,1.0,1.0,NaN,NaN,101162.0,NaN,NaN,NaN
1087,101464,UNO Charter School - Campus 18,NaN,001U000000oGO1q,NaN,NaN,Chicago,IL,NaN,NaN,www.ucsnschools.org,NaN,NaN,NaN,NaN,City of Chicago SD 299,1155,NaN,Charter,Charter,Not yet open,UNO Charter School Network,NaN,NaN,NaN,NaN,ALL,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1088,101466,UNO Charter School - Campus 17,NaN,001U000000oGO1g,NaN,NaN,Chicago,IL,NaN,NaN,www.ucsnschools.org,NaN,NaN,NaN,NaN,City of Chicago SD 299,1155,NaN,Charter,Charter,Not yet open,UNO Charter School Network,NaN,NaN,NaN,NaN,ALL,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1098,101487,Intrinsic - Campus 2,NaN,NaN,NaN,NaN,Chicago,IL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,City of Chicago SD 299,1155,NaN,Charter,NaN,Not yet open,Intrinsic Schools,NaN,NaN,NaN,NaN,ALL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#create df
sen_dist_charter = pd.DataFrame(charter_schools.State_Sen_District.value_counts())
sen_dist_charter.reset_index(inplace=True)
sen_dist_charter.columns = ['district', 'total_charters']
sen_dist_charter.district = sen_dist_charter.district.astype('int')
sen_dist_chart_legis = sen_dist_charter.merge(legislator, how='inner', on=None, left_on='district', 
                      right_on='District')

In [11]:
#who are the schools?

#join senate table
chart_legis = charter_schools.merge(legislator, how='inner', on=None, left_on='State_Sen_District', 
                      right_on='District')

#who represents each school?
chart_legis[['District_Name', "School_Type_Rollup", "In_Chicago", "GEOID10", 
             "District", "Legislator", "Party", "roll_call_vote", 'Lat', 'Long']] #.to_csv('../outcomes/INCS-task/chart_legis.csv')


,District_Name,School_Type_Rollup,In_Chicago,GEOID10,District,Legislator,Party,roll_call_vote,Lat,Long
0,Normal CUSD 5,Charter,0.0,NaN,53,Jason A. Barickman,R,N,40.498016,-89.035817
1,City of Chicago SD 299,Charter,1.0,1.703121e+10,20,Iris Y. Martinez,D,N,41.937654,-87.727053
2,City of Chicago SD 299,Charter,1.0,1.703121e+10,20,Iris Y. Martinez,D,N,41.937364,-87.726682
3,City of Chicago SD 299,Charter,1.0,1.703114e+10,20,Iris Y. Martinez,D,N,41.966403,-87.721318
4,City of Chicago SD 299,Charter,1.0,1.703116e+10,20,Iris Y. Martinez,D,N,41.950623,-87.710463
5,City of Chicago SD 299,Charter,1.0,1.703121e+10,20,Iris Y. Martinez,D,N,41.937471,-87.699245
6,City of Chicago SD 299,Charter,1.0,1.703116e+10,20,Iris Y. Martinez,D,N,41.939099,-87.740081
7,City of Chicago SD 299,Charter,1.0,1.703121e+10,20,Iris Y. Martinez,D,N,41.935695,-87.718096
8,City of Chicago SD 299,Charter,1.0,1.703156e+10,1,Antonio Muñoz,D,P,41.807575,-87.739752
9,City of Chicago SD 299,Charter,1.0,1.703159e+10,1,Antonio Muñoz,D,P,41.825826,-87.667748


# What Senate Districts have students who attend a Charter School?

In [12]:
#student data table (only includes Chicago) and merge to schools on school info id
student_school = student.merge(school, how='inner', left_on='School_Info_Id', right_on='School_Info_ID')
student_school = student_school.merge(legislator, how='inner', left_on='State_Sen_District', right_on='District')

In [13]:
#what percent of students in CPS go to charter schools (15% --> 57,699 students)
student_school.School_Type_Rollup.value_counts()

Non-selective    286619
Charter          57699 
Selective        33608 
Non-Selective    1738  
Name: School_Type_Rollup, dtype: int64

In [14]:
#charter mask
charter_students = student_school[student_school.School_Type_Rollup == 'Charter']

#adding total number of charter school students
legislator = legislator.merge(pd.DataFrame(charter_students.groupby('District').Stu_GEOID10.count()), how='outer', on='District', )

In [15]:
#adding the number of charter schools (school_student.School_Type_Rollup == 'Charter')
legislator = legislator.merge(sen_dist_charter, how='outer', left_on='District', right_on='district').drop('district', axis=1)

# What Senate Districts (in Chicago) have "low-performing" schools?

In [16]:
# >=3.0 (good standing), <2.9 and >2.0 (provisional support), <2.0 (intensive support)

school_qual = school.merge(qual, how='inner', left_on='School_Info_ID', right_on='School_Info_ID')

support_schools = school_qual[school_qual.Total_Points_Earned < 3.0].groupby('State_Sen_District').Total_Points_Earned.count()

In [17]:
#schools earning 3.0+ points counted by District
strong_schools = school_qual[school_qual.Total_Points_Earned >= 3.0].groupby('State_Sen_District').Total_Points_Earned.count()

In [18]:
#appended to legislator table
legislator = legislator.merge(pd.DataFrame(support_schools), how='outer', left_on='District', right_on='State_Sen_District')

In [19]:
#appended to legislator table
legislator = legislator.merge(pd.DataFrame(strong_schools), how='outer', left_on='District', right_on='State_Sen_District')

In [20]:
legislator.columns

Index(['Legislator_Database_ID', 'District', 'Legislator', 'Party', 'Chamber',
       'Assembly_Number', 'roll_call_vote', 'target', 'Stu_GEOID10',
       'total_charters', 'Total_Points_Earned_x', 'Total_Points_Earned_y'],
      dtype='object')

# Two-Way Frequencies

In [21]:
#cleaning for frequency analysis
legislator.columns = ['Legislator_Database_ID', 'District', 'Legislator', 'Party', 'Chamber',
                      'Assembly_Number', 'roll_call_vote', 'target', 'total_charter_students', 
                      'total_charter_schools', 'total_support_schools', 'total_good_stand_schools']

#legislator.to_csv('../outcomes/INCS-task/legislator_chars.csv')
legislator.total_good_stand_schools.fillna(-100, inplace=True)
legislator.total_support_schools.fillna(-100, inplace=True)
legislator.total_charter_students.fillna(-100, inplace=True)
legislator.total_charter_schools.fillna(0, inplace=True)

#fix true 0s
legislator.total_support_schools.iloc[7] = 0
legislator.total_support_schools.iloc[26] = 0

#make total measurable cps schools columns
legislator['total_cps'] = legislator.total_support_schools + legislator.total_good_stand_schools

#make good/support ratio
legislator['good_support_ratio'] = legislator.total_support_schools/legislator.total_good_stand_schools

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [22]:
#masking
yay_mask = legislator.roll_call_vote == 'Y'
no_charter_schools_mask = legislator.total_charter_schools < 1
charter_schools_mask = legislator.total_charter_schools > 0
rep_mask = legislator.Party == 'R'
dem_mask = legislator.Party == 'D'
four_charter_schools_mask = legislator.total_charter_schools < 4

In [23]:
#P(voting Nay) (29%)
legislator.roll_call_vote.value_counts(normalize=True)

Y     0.559322
N     0.288136
NV    0.118644
P     0.033898
Name: roll_call_vote, dtype: float64

In [37]:
#P(voting Nay & being a Republican) (68%)
legislator[rep_mask].roll_call_vote.value_counts(normalize=False)

N     15
Y     5 
NV    2 
Name: roll_call_vote, dtype: int64

In [31]:
#P(voting Nay & being a Democrat) (5%)
legislator[dem_mask].roll_call_vote.value_counts(normalize=True)

Y     0.756757
NV    0.135135
N     0.054054
P     0.054054
Name: roll_call_vote, dtype: float64

In [25]:
#P(voting)
legislator[no_charter_schools_mask].roll_call_vote.value_counts(normalize=True)

Y     0.56250
N     0.34375
NV    0.09375
Name: roll_call_vote, dtype: float64

In [110]:
#P(voting Nay &  having few charters) --> 32%
legislator[four_charter_schools_mask].roll_call_vote.value_counts(normalize=True)

Y     0.574468
N     0.319149
NV    0.106383
Name: roll_call_vote, dtype: float64

In [26]:
#P(voting Nay & having charter schools) (22%)
#P(voting Nay & not having charter schools) (34%)
legislator[no_charter_schools_mask].roll_call_vote.value_counts(normalize=True)

Y     0.56250
N     0.34375
NV    0.09375
Name: roll_call_vote, dtype: float64

In [109]:
#P(voting Nay & being Rep & not having charters) --> 61%
legislator[rep_mask & no_charter_schools_mask].roll_call_vote.value_counts(normalize=True)

Y     0.574468
N     0.319149
NV    0.106383
Name: roll_call_vote, dtype: float64

In [29]:
#Strategy #1 --> target Republicans (who also don't have charter schools in their districts)
legislator[(rep_mask)]

,Legislator_Database_ID,District,Legislator,Party,Chamber,Assembly_Number,roll_call_vote,target,total_charter_students,total_charter_schools,total_support_schools,total_good_stand_schools,total_cps,good_support_ratio
20,10042,21,Michael Connelly,R,Senate,100,N,1,-100.0,0.0,-100.0,-100.0,-200.0,1.0
23,10048,24,Chris Nybo,R,Senate,100,N,1,-100.0,0.0,-100.0,-100.0,-200.0,1.0
24,10050,25,Jim Oberweis,R,Senate,100,N,1,-100.0,0.0,-100.0,-100.0,-200.0,1.0
25,10052,26,Dan McConchie,R,Senate,100,N,1,-100.0,0.0,-100.0,-100.0,-200.0,1.0
26,10054,27,Tom Rooney,R,Senate,100,N,1,210.0,1.0,0.0,1.0,1.0,0.0
31,10064,32,Pamela J. Althoff,R,Senate,100,NV,0,-100.0,0.0,-100.0,1.0,-99.0,-100.0
32,10066,33,Karen McConnaughay,R,Senate,100,N,1,-100.0,1.0,-100.0,-100.0,-200.0,1.0
34,10070,35,Dave Syverson,R,Senate,100,N,1,-100.0,1.0,-100.0,-100.0,-200.0,1.0
35,10072,36,Neil Anderson,R,Senate,100,Y,0,-100.0,0.0,-100.0,-100.0,-200.0,1.0
36,10074,37,Chuck Weaver,R,Senate,100,N,1,-100.0,0.0,-100.0,-100.0,-200.0,1.0


In [30]:
#Strategy #2 --> chicago focus
chicago_mask = legislator.total_charter_students != -100

charter_students_mask = legislator.total_charter_students > 3000
low_performing_mask = legislator.total_support_schools > 0
high_performing_mask = legislator.total_good_stand_schools > 0
few_charters_mask = legislator.total_charter_schools < 5
good_support_median = legislator.good_support_ratio > legislator.good_support_ratio[chicago_mask].median() #.13
good_support_mean = legislator.good_support_ratio > legislator.good_support_ratio[chicago_mask].mean() #.24

In [31]:
#P(voting nay and being in CPS) --> 18%
legislator[chicago_mask].roll_call_vote.value_counts(normalize=True)

Y     0.529412
N     0.176471
NV    0.176471
P     0.117647
Name: roll_call_vote, dtype: float64

In [32]:
#P(voting nay and being Repub and being in CPS) --> 100% (there are no other repubs to flip in Chi)
legislator[chicago_mask & rep_mask].roll_call_vote.value_counts(normalize=True)

N    1.0
Name: roll_call_vote, dtype: float64

In [33]:
#P(voting Nay in Chi and having a low_performing school) --> 13%
legislator[chicago_mask & low_performing_mask].roll_call_vote.value_counts(normalize=True)

Y     0.533333
NV    0.200000
P     0.133333
N     0.133333
Name: roll_call_vote, dtype: float64

In [34]:
#P(voting Nay in Chi & having high performing schools) --> 18%
legislator[chicago_mask & high_performing_mask].roll_call_vote.value_counts(normalize=True) 

Y     0.529412
N     0.176471
NV    0.176471
P     0.117647
Name: roll_call_vote, dtype: float64

In [35]:
#P(voting Nay in Chi & having charter students) --> 18%
#P(voting Nay in Chi & having >2k charter students) --> 29%
legislator[chicago_mask & charter_students_mask].roll_call_vote.value_counts(normalize=True) 

Y    0.571429
N    0.285714
P    0.142857
Name: roll_call_vote, dtype: float64

In [37]:
#P(voting Nay in Chi & having less that 5 charter schools) --> 22%
legislator[chicago_mask & few_charters_mask].roll_call_vote.value_counts(normalize=True) 

Y    0.555556
P    0.222222
N    0.222222
Name: roll_call_vote, dtype: float64

In [64]:
#P(voting Nay in Chicago & having a higher support_good ratio >median) #13%
legislator[chicago_mask & good_support_median].roll_call_vote.value_counts(normalize=True) 

#P(voting Nay in Chicago & having a higher support_good ratio >mean) #.2%
legislator[chicago_mask & good_support_mean].roll_call_vote.value_counts(normalize=True) 

Y     0.6
NV    0.2
N     0.2
Name: roll_call_vote, dtype: float64

In [113]:
#legislator.to_csv('./INCS-task/legislator_chars.csv')

In [39]:
##P(voting Nay & having >3k students in a charter) (18%)
legislator[charter_students_mask].roll_call_vote.value_counts(normalize=True)

Y    0.571429
N    0.285714
P    0.142857
Name: roll_call_vote, dtype: float64

# Modeling

In [40]:
#modeling w/all Districts included
from sklearn.linear_model import LogisticRegression

binary = legislator
binary.replace(-100, 0, inplace=True)
binary.replace(-200, 0, inplace=True)
binary.fillna(0, inplace=True)

X = pd.get_dummies(binary[['Party', 'total_charter_students', 'total_charter_schools', 'total_support_schools', 
                           'total_good_stand_schools', 'good_support_ratio']], columns=['Party'], drop_first=True)

#0 = yay & et al, 1 = nay (we are trying to move yays to nays)
y = binary.target

In [41]:
lr = LogisticRegression()
lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [42]:
binary['target'] = y

In [43]:
binary['prediction'] = lr.predict(X)

In [44]:
binary['prob_yay_0'] = lr.predict_proba(X)[:,0]

In [45]:
binary['prob_nay_1'] = lr.predict_proba(X)[:,1]

In [46]:
X.columns, lr.coef_

(Index(['total_charter_students', 'total_charter_schools',
        'total_support_schools', 'total_good_stand_schools',
        'good_support_ratio', 'Party_R'],
       dtype='object'),
 array([[ 1.53211077e-03, -3.57942318e-02, -2.33583166e-01,
         -1.62494245e-01, -6.47494946e-01,  1.97375916e+00]]))

In [48]:
#observations from coefs:
#most predicitive is party
#other values are very small

In [103]:
#Chicago Focus, small sample size, very unbalanced classes.
cps_mask = legislator.total_cps > 1

chicago = legislator[cps_mask]
chicago.replace(-100, 0, inplace=True)
chicago.replace(-200, 0, inplace=True)
chicago.fillna(0, inplace=True)

X = pd.get_dummies(chicago[['Party', 'total_charter_students', 'total_charter_schools',
                            'total_support_schools', 'total_good_stand_schools', 'good_support_ratio'
                            ]], columns=['Party'], drop_first=True)

#0 = yay & et al, 1 = nay (we are trying to move yays to nays)
y = chicago.target

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [104]:
lr_chi = LogisticRegression()
lr_chi.fit(X, y)

chicago['target'] = y

chicago['prediction'] = lr_chi.predict(X)

X.columns, lr_chi.coef_

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(Index(['total_charter_students', 'total_charter_schools',
        'total_support_schools', 'total_good_stand_schools',
        'good_support_ratio'],
       dtype='object'),
 array([[ 0.00174998, -0.0534389 , -0.27388874, -0.19444686, -0.37436456]]))

In [105]:
chicago['prob_yay_0'] = lr_chi.predict_proba(X)[:,0]
chicago['prob_nay_1'] = lr_chi.predict_proba(X)[:,1]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [107]:
chicago.sort_values('prob_nay_1', ascending=False)

,Legislator_Database_ID,District,Legislator,Party,Chamber,Assembly_Number,roll_call_vote,target,total_charter_students,total_charter_schools,total_support_schools,total_good_stand_schools,total_cps,good_support_ratio,prediction,prob_yay_0,prob_nay_1
4,10010,5,Patricia Van Pelt,D,Senate,100,N,1,13751.0,30.0,23.0,70.0,93.0,0.328571,1,0.124135,0.875865
1,10004,2,Omar Aquino,D,Senate,100,Y,0,6204.0,12.0,2.0,47.0,49.0,0.042553,1,0.490080,0.509920
14,10030,15,Napoleon Harris III,D,Senate,100,NV,0,656.0,2.0,4.0,3.0,7.0,1.333333,0,0.833430,0.166570
11,10024,12,Steven M. Landek,D,Senate,100,Y,0,0.0,0.0,1.0,5.0,6.0,0.200000,0,0.857440,0.142560
17,10036,18,Bill Cunningham,D,Senate,100,Y,0,497.0,1.0,1.0,10.0,11.0,0.100000,0,0.871321,0.128679
2,10006,3,Mattie Hunter,D,Senate,100,Y,0,8339.0,18.0,24.0,44.0,68.0,0.545455,0,0.898003,0.101997
6,10014,7,Heather A. Steans,D,Senate,100,NV,0,2024.0,4.0,2.0,23.0,25.0,0.086957,0,0.900034,0.099966
19,10040,20,Iris Y. Martinez,D,Senate,100,N,1,3583.0,7.0,3.0,35.0,38.0,0.085714,0,0.903479,0.096521
38,10078,39,Don Harmon,D,Senate,100,Y,0,0.0,0.0,3.0,3.0,6.0,1.000000,0,0.904872,0.095128
7,10016,8,Ira I. Silverstein,D,Senate,100,Y,0,908.0,1.0,0.0,18.0,18.0,0.000000,0,0.919657,0.080343


In [111]:
#Strategy 1 --> target

legislator[rep_mask & (legislator.target == 0) & (legislator.roll_call_vote != "N")]#.to_csv('./strategy1.csv')

In [112]:
#Strategy 2 --> Chicago
ratio_mask = chicago.good_support_ratio > .13

chicago[cps_mask].sort_values('prob_nay_1', ascending=False)#.to_csv('./strategy2.csv')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
